<a href="https://colab.research.google.com/github/serozhenko/A-B-test/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import scipy.stats as stats

import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Mate_Academy

/content/drive/MyDrive/Mate_Academy


In [ ]:
df = pd.read_csv("a_b_test.csv")
df

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,session_with_orders,1
1,2020-11-03,New Zealand,desktop,Oceania,Direct,2,2,session_with_orders,1
2,2020-11-04,Bulgaria,mobile,Europe,Paid Search,2,1,session_with_orders,1
3,2020-11-04,Kuwait,mobile,Asia,Organic Search,2,2,session_with_orders,1
4,2020-11-05,Serbia,desktop,Europe,Social Search,2,2,session_with_orders,1
...,...,...,...,...,...,...,...,...,...
800991,2020-12-19,Vietnam,desktop,Asia,Organic Search,3,2,view_search_results,1
800992,2021-01-10,Vietnam,mobile,Asia,Direct,4,1,first_visit,1
800993,2020-11-24,Vietnam,desktop,Asia,Direct,3,2,view_promotion,1
800994,2020-12-11,Vietnam,desktop,Asia,Paid Search,4,1,view_item,1


In [ ]:
# Перевірка унікальності значень стовпчика event_name для подальших розрахунків!!!
result = []
for item in df['event_name']:
    if item not in result:
        result.append(item)

print(result)

In [ ]:
import pandas as pd
import statsmodels.api as sm



#Метрики
metrics = {
    'add_payment / session': ('add_payment_info', 'session'),
    'add_shipping / session': ('add_shipping_info', 'session'),
    'begin_checkout / session': ('begin_checkout', 'session'),
    'new_accounts / session': ('account', 'session')
}

results = []

# Рахуємо по кожному тесту
for test_number in df['test'].unique():
    test_data = df[df['test'] == test_number]

    for metric_name, (numerator_event, denominator_event) in metrics.items():
        # Тестова група (group 2)
        numerator_ev = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 2)]['value'].sum()
        denominator_ev = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 2)]['value'].sum()

        # Контрольна група (group 1)
        numerator_cc = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 1)]['value'].sum()
        denominator_cc = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 1)]['value'].sum()

        # Перевірка на валідність знаменників
        if denominator_ev > 0 and denominator_cc > 0:
            # Конверсії
            conv_rate_ev = numerator_ev / denominator_ev
            conv_rate_cc = numerator_cc / denominator_cc

            # Зміна метрики у %
            metric_change = (conv_rate_ev - conv_rate_cc) / conv_rate_cc

            # Z-test
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_ev, numerator_cc],
                [denominator_ev, denominator_cc]
            )

            # Додаємо у результати
            results.append({
                "test_number": test_number,
                "metric": metric_name,
                "numerator_ev": numerator_ev,
                "denominator_ev": denominator_ev,
                "conversion_rate_ev": conv_rate_ev,
                "numerator_cc": numerator_cc,
                "denominator_cc": denominator_cc,
                "conversion_rate_cc": conv_rate_cc,
                "metric_change": metric_change,
                "z_stat": z_stat,
                "p_value": p_value,
                "significant": p_value < 0.05
            })

final_df = pd.DataFrame(results)

final_df = final_df.sort_values(by=["test_number", "metric"]).reset_index(drop=True)


final_df

,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_cc,denominator_cc,conversion_rate_cc,metric_change,z_stat,p_value,significant
0,1,add_payment / session,2229,45193,0.049322,1988,45362,0.043825,0.125420,3.924884,0.000087,True
1,1,add_shipping / session,3221,45193,0.071272,3034,45362,0.066884,0.065605,2.603571,0.009226,True
2,1,begin_checkout / session,4021,45193,0.088974,3784,45362,0.083418,0.066606,2.978783,0.002894,True
3,1,new_accounts / session,3681,45193,0.081451,3823,45362,0.084278,-0.033543,-1.542883,0.122859,False
4,2,add_payment / session,2409,50244,0.047946,2344,50637,0.046290,0.035769,1.240994,0.214608,False
5,2,add_shipping / session,3510,50244,0.069859,3480,50637,0.068724,0.016510,0.709557,0.477979,False
6,2,begin_checkout / session,4313,50244,0.085841,4262,50637,0.084168,0.019882,0.952898,0.340642,False
7,2,new_accounts / session,4184,50244,0.083274,4165,50637,0.082252,0.012419,0.588793,0.556000,False
8,3,add_payment / session,3697,70439,0.052485,3623,70047,0.051722,0.014746,0.643172,0.520112,False
9,3,add_shipping / session,5188,70439,0.073652,5298,70047,0.075635,-0.026212,-1.413727,0.157442,False


In [ ]:
# Зберігаємо результати в CSV файл
final_df.to_csv('a_b_test_results.csv', index=False)
from google.colab import files
files.download("a_b_test_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## [**Посилання на дашборд**](https://public.tableau.com/app/profile/viktoriia.serozhenko6318/viz/ABTest_17552153375840/AB_Test?publish=yes)